# Regression Party - K-Nearest Neighbors
Denna notebook täcker en analys av datan i hemnet_data_clean.csv samt en regression med hjälp av K-Nearest Neighbors.

## Importerar bibliotek

In [1]:
import pandas as pd

## Importerar data

In [2]:
hemnet_data = pd.read_csv('hemnet_data_clean.csv')

## Förbehandling av data

### Analys
Vår data har 500 datapunkter. Vi har några nullvärden i vissa kolumner som vi kommer behöva ta bort eller ersätta med något sorts snittvärde. Det finns även vissa dubbletter när vi räknar bor kolumnen 'Unnamed: 0'. Dessa bör tas bort.

Så här bör varje kolumn bearbetas:
* Unnamed: 0 - Detta är mest troligtvis ett indexvärde och kan tas bort.
* address - Detta värde blir svårt att konvertera numeriskt och kan tas bort.
* asked_price - Detta värde behålls i vårat X.
* coordinate - Detta värde behöver delas upp och kan användas i vårat X som 'latitude' och 'longitude'.
* final_price - Detta värde blir vår y serie.
* pourcentage_difference - Detta värde bör vara baserat på 'asked_price' och 'final_price' och tas därför bort.
* land_area - Detta värde behålls i vårat X.
* area - Detta värde behålls i vårat X.
* commune - Detta värde blir svårt att konvertera numeriskt och kan tas bort.
* price_per_area - Detta värde bör vara baserat på 'area' och antingen 'asked_price' eller 'final_price' och tas därför bort.
* rooms - Detta värde behålls i vårat X.
* sale_date - Detta värde kan användas om vi vill justera priserna baserat på inflation, men bör sen strykas från modellen.
* supplemental_area - Detta värde behålls i vårat X.

In [14]:
display(hemnet_data.loc[:,'address':].value_counts())

address                  asked_price  coordinate                                final_price  pourcentage_difference  land_area  area   commune                                      price_per_area  rooms  sale_date        supplemental_area
Drottningholmsvägen 556  8750000.0    [59.33139, 17.92017]                      9200000      5.0                     475.0      95.0   Bromma Nockebyhov, Stockholms kommun         96842.0         5.0    Såld 2022-06-15  49.0                 2
Tallvägen 1A             11950000.0   [59.4156876, 17.9522781]                  11300000     5.0                     1134.0     210.0  Helenelund, Sollentuna kommun                53810.0         9.0    Såld 2022-06-10  60.0                 2
Markviksvägen 15         6475000.0    [59.371685, 17.8250389]                   6950000      7.0                     635.0      117.0  Hässelby Södra Villastad, Stockholms kommun  59402.0         5.0    Såld 2022-06-17  44.0                 2
Vinstavägen 14           7875000.

In [3]:
display(hemnet_data.head())

,Unnamed: 0,address,asked_price,coordinate,final_price,pourcentage_difference,land_area,area,commune,price_per_area,rooms,sale_date,supplemental_area
0,0.0,Markviksvägen 15,6475000.0,"[59.371685, 17.8250389]",6950000,7.0,635.0,117.0,"Hässelby Södra Villastad, Stockholms kommun",59402.0,5.0,Såld 2022-06-17,44.0
1,1.0,Mälarhöjdsvägen 38,9950000.0,"[59.3008766, 17.9478054]",11300000,14.0,882.0,195.0,"Mälarhöjden - Hägersten, Stockholms kommun",57949.0,5.0,Såld 2022-06-17,22.0
2,2.0,Häradsvägen 202,5795000.0,"[59.27202322587505, 17.954076342308372]",6200000,7.0,1224.0,90.0,"Segeltorp, Huddinge kommun",68889.0,6.0,Såld 2022-06-17,90.0
3,3.0,Hemmansvägen 18,11950000.0,"[59.28390406370304, 18.07625832074728]",12700000,6.0,542.0,110.0,"Gamla Enskede, Stockholms kommun",115455.0,5.0,Såld 2022-06-17,48.0
4,4.0,Alholmsbacken 8,7950000.0,"[59.28134, 17.9009438]",8100000,2.0,690.0,157.0,"Skärholmshöjden., Stockholms kommun",51592.0,6.0,Såld 2022-06-17,0.0


In [4]:
display(hemnet_data.describe())

,Unnamed: 0,asked_price,final_price,pourcentage_difference,land_area,area,price_per_area,rooms,supplemental_area
count,500.000000,5.000000e+02,5.000000e+02,500.000000,495.000000,499.000000,499.000000,490.000000,500.000000
mean,24.950000,9.375150e+06,1.004715e+07,8.914000,1012.278788,142.759519,79120.142285,6.165306,55.460000
std,14.448183,3.911988e+06,4.014927e+06,8.797579,1508.117107,116.952734,29949.142598,2.292341,54.933193
min,0.000000,1.475000e+06,1.490000e+06,0.000000,93.000000,21.000000,20617.000000,2.000000,0.000000
25%,12.475000,6.995000e+06,7.450000e+06,1.000000,557.500000,102.500000,55239.000000,5.000000,22.000000
50%,24.950000,8.475000e+06,8.852500e+06,7.000000,729.000000,134.000000,71635.000000,6.000000,54.000000
75%,37.425000,1.012375e+07,1.130000e+07,14.000000,977.000000,162.000000,99537.000000,7.000000,79.000000
max,49.900000,2.990000e+07,3.110000e+07,48.000000,13811.000000,1795.000000,187500.000000,45.000000,845.000000


In [8]:
display(hemnet_data.notnull().all())

Unnamed: 0                 True
address                    True
asked_price                True
coordinate                 True
final_price                True
pourcentage_difference     True
land_area                 False
area                      False
commune                    True
price_per_area            False
rooms                     False
sale_date                  True
supplemental_area          True
dtype: bool